In [1]:
from jupyter_dash import JupyterDash

In [2]:
# https://dash-bootstrap-components.opensource.faculty.ai/examples/iris/
# https://dash-bootstrap-components.opensource.faculty.ai/docs/components/form/

import dash
import dash_html_components as html
import dash_core_components as dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
from datetime import datetime as dt
from datetime import timedelta
import re
#from plotly.subplots import make_subplots


previous_days = 5
money_ini = 1000
bitcoin_ini = 0.1
prop_ini = 0.05

path = 'C:/Users/Victor.Maton/Documents/bitcoin/'
data_name = 'bitcoin_data'

df = pd.read_csv(path+data_name+'.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.rename(columns={'Close': 'Closing Price'}, inplace=True)
df.set_index('Date', inplace=True)
df = df.squeeze('columns')

date_first = df.index[0]
date_last = df.index[-1]

starting_date = date_first + timedelta(days=previous_days)

dff = df.loc[date_first:date_last]

money = money_ini
bitcoin = bitcoin_ini
money_vec = []
bitcoin_vec = []
worth_vec = []

for date in dff.loc[date_first:starting_date].index:
    current_price = dff.loc[date]

    worth = money + bitcoin*current_price # Total Worth in Money

    if(date != starting_date):
        money_vec += [money]
        bitcoin_vec += [bitcoin]
        worth_vec += [worth]

for date in dff.loc[starting_date:].index:
    current_price = dff.loc[date]
    date_prior = date - timedelta(days=previous_days)
    prices_previous = dff.loc[date_prior:date] # Last prices (current included)

    if(all(current_price > prices_previous[:-1].values)):   # If current bitcoin price higher I sell some 
        money = money + bitcoin*prop_ini*current_price       # I have more money
        bitcoin = bitcoin - prop_ini*bitcoin                 # I have less bitcoin

    if(all(current_price < prices_previous[:-1].values)):   # If current bitcoin price lower I buy some
        bitcoin = bitcoin + (money*prop_ini)/current_price   # I have more bitcoin
        money = money - money*prop_ini                       # I have less money
        
    worth = money + bitcoin*current_price # Total Worth in Money
    
    money_vec += [money]
    bitcoin_vec += [bitcoin]
    worth_vec += [worth]

data_simu = {'Closing Price': dff, 'Money' : money_vec, 'Bitcoin' : bitcoin_vec, 'Total Worth' : worth_vec } 
df_simu = pd.DataFrame(data_simu)

df_simu['Relative Closing Price'] = df_simu['Closing Price']/df_simu['Closing Price'][0]
df_simu['Relative Total Worth'] = df_simu['Total Worth']/df_simu['Total Worth'][0]
df_simu['Relative Time'] = [x/(len(dff)-1) for x in [*range(len(df_simu))]]

fig2 = px.scatter(df_simu, 
                    x='Relative Closing Price', 
                    y='Relative Total Worth',
                    color='Relative Time', 
                    color_continuous_scale=px.colors.sequential.Viridis,
                    title='Performance against "Buy and Hold" across selected period')
line = df_simu['Relative Closing Price'] if df_simu['Relative Closing Price'].max() < df_simu['Relative Total Worth'].max() else df_simu['Relative Total Worth']
fig2.add_scatter(x=line, y=line, mode='lines', opacity=0.5, name='Unitary reference')
fig2.update(layout_showlegend=False)

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP]) #VIC
#app = dash.Dash()

graph_ts = dcc.Graph(
    id = 'output-timeseries',
    figure = px.line(dff, 
                    x=dff.index, 
                    y='Closing Price', 
                    title='Bitcoin closing price across selected period'
    ).update_xaxes(rangeslider_visible=True)
)

graph_worth = dcc.Graph(
        id='output-simu',
        figure=px.line(df_simu,
            x=df_simu.index, 
            y='Total Worth').update_xaxes(rangeslider_visible=True)
)

graph_perf = dcc.Graph(
        id='output-simu2',
        figure=fig2
)

dropdown = dcc.Dropdown(
        id='label-simu',
        options=[
            {'label': 'Total Worth', 'value': 'Total Worth'},
            {'label': 'Money', 'value': 'Money'},
            {'label': 'Bitcoin', 'value': 'Bitcoin'}
        ],
        value='Total Worth'
)

filter_date = dcc.DatePickerRange(
                        id='my-date-picker-range',
                        min_date_allowed=date_first,
                        max_date_allowed=date_last,
                        initial_visible_month=date_last,
                        start_date=date_first,
                        end_date=date_last
)

filter_money = dbc.Input(
                        id='num-money',
                        type='number',
                        value=money_ini
)

filter_bitcoin = dbc.Input(
                    id='num-bitcoin',
                    type='number',
                    value=bitcoin_ini
)

filter_days = dbc.Input(
                        id='num-days',
                        type='number',
                        value=previous_days
)

filter_prop = dbc.Input(
                        id='prop_sellbuy',
                        type='number',
                        value=prop_ini
)

content = [
    dbc.Row(
            dbc.Col(
                [
                    graph_ts
                ],
                width=12
            )
    ),
    dbc.Row(
        dbc.Col(
                        [
                            #html.Label('Period of interest :'),
                            filter_date
                        ],
                        width={"size": 8, "offset": 4},
        )  
    ),
    dbc.Row(
            [
                dbc.Col(
                    [
                        html.Label('Initial Money'),
                        filter_money
                    ],
                    md=3
                ),
                dbc.Col(
                    [
                        html.Label('Initial Bitcoin'),
                        filter_bitcoin
                    ],
                    md=3
                ),
                dbc.Col(
                    [
                        html.Label('Last N° Days for transaction'),
                        filter_days
                    ],
                    md=3
                ),
                dbc.Col(
                    [
                        html.Label('Transaction Proportion'),
                        filter_prop
                    ],
                    md=3
                ),
            ]
    ),
    dbc.Row(
        dbc.Col(
            [
                dropdown
            ]
        )
    ),
    dbc.Row(
        dbc.Col(
            [
                graph_worth
            ]
        )
    ),
    dbc.Row(
        dbc.Col(
            [
                graph_perf
            ]
        )
    )
]

app.layout = dbc.Container(
    content,
    fluid = True
)

@app.callback(
    dash.dependencies.Output('output-timeseries', 'figure'),
    [dash.dependencies.Input('my-date-picker-range', 'start_date'),
     dash.dependencies.Input('my-date-picker-range', 'end_date')])
def update_output_ts(start_date, end_date):
    df_upd = dff.loc[start_date:end_date]

    fig = px.line(df_upd, 
                    x=df_upd.index, 
                    y='Closing Price', 
                    #color_discrete_sequence=['goldenrod'],
                    title='Bitcoin closing price across selected period').update_xaxes(rangeslider_visible=True)

    fig.update_xaxes(range=[df_upd.index.min(), df_upd.index.max()])

    fig.update_yaxes(range=[
        df_upd.min() - 0.05*df_upd.min() if df_upd.min() - 0.05*df_upd.min() >= 0 else 0, 
        df_upd.max() + 0.05*df_upd.max()])

    return fig

@app.callback(
    [dash.dependencies.Output('output-simu', 'figure'),
    dash.dependencies.Output('output-simu2', 'figure')],
    [dash.dependencies.Input('my-date-picker-range', 'start_date'),
     dash.dependencies.Input('my-date-picker-range', 'end_date'),
     dash.dependencies.Input('num-money', 'value'),
     dash.dependencies.Input('num-bitcoin', 'value'),
     dash.dependencies.Input('num-days', 'value'),
     dash.dependencies.Input('label-simu', 'value'),
     dash.dependencies.Input('prop_sellbuy', 'value')])
def update_output_simu(start_date, end_date, money, bitcoin, previous_days, label_simu, prop):
    start_date = dt.strptime(re.split('T| ', start_date)[0], '%Y-%m-%d')
    end_date = dt.strptime(re.split('T| ', end_date)[0], '%Y-%m-%d')
    starting_date = start_date + timedelta(days=round(previous_days))

    df_upd = dff.loc[start_date:end_date]

    money_vec = []
    bitcoin_vec = []
    worth_vec = []

    for date in df_upd.loc[start_date:starting_date].index:
        current_price = df_upd.loc[date]

        worth = money + bitcoin*current_price # Total Worth in Money

        if(date != starting_date):
            money_vec += [money]
            bitcoin_vec += [bitcoin]
            worth_vec += [worth]

    for date in df_upd.loc[starting_date:].index:
        current_price = df_upd.loc[date]
        date_prior = date - timedelta(days=previous_days)
        prices_previous = df_upd.loc[date_prior:date] # contains current price

        if(all(current_price > prices_previous[:-1].values)):  # If current bitcoin price higher I sell some 
            money = money + bitcoin*prop*current_price      # I have more money
            bitcoin = bitcoin - prop*bitcoin                # I have less bitcoin

        if(all(current_price < prices_previous[:-1].values)):  # If current bitcoin price lower I buy some
            bitcoin = bitcoin + (money*prop)/current_price  # I have more bitcoin
            money = money - money*prop                      # I have less money

        worth = money + bitcoin*current_price # Total Worth in Money
        
        money_vec += [money]
        bitcoin_vec += [bitcoin]
        worth_vec += [worth]

    data_simu = {'Closing Price': df_upd, 'Money' : money_vec, 'Bitcoin' : bitcoin_vec, 'Total Worth' : worth_vec} 
    df_simu = pd.DataFrame(data_simu)

    fig = px.line(df_simu, 
                    x=df_simu.index, 
                    y=label_simu, 
                    #color_discrete_sequence=['goldenrod'],
                    title=label_simu + ' across selected period').update_xaxes(rangeslider_visible=True)
    fig.update_xaxes(range=[df_simu.index.min(), df_simu.index.max()])

    fig.update_yaxes(range=[
        df_simu[label_simu].min() - 0.05*df_simu[label_simu].min() if df_simu[label_simu].min() - 0.05*df_simu[label_simu].min() >= 0 else 0, 
        df_simu[label_simu].max() + 0.05*df_simu[label_simu].max()])

    df_simu['Relative Closing Price'] = df_simu['Closing Price']/df_simu['Closing Price'][0]
    df_simu['Relative Total Worth'] = df_simu['Total Worth']/df_simu['Total Worth'][0]
    df_simu['Relative Time'] = [x/(len(df_simu)-1) for x in [*range(len(df_simu))]]

    fig2 = px.scatter(df_simu, 
                    x='Relative Closing Price', 
                    y='Relative Total Worth',
                    color='Relative Time', 
                    color_continuous_scale=px.colors.sequential.Viridis,
                    title='Performance against "Buy and Hold" across selected period')
    line = df_simu['Relative Closing Price'] if df_simu['Relative Closing Price'].max() < df_simu['Relative Total Worth'].max() else df_simu['Relative Total Worth']
    fig2.add_scatter(x=line, y=line, mode='lines', opacity=0.5, name='Unitary reference')
    fig2.update(layout_showlegend=False)
    #fig2.add_scatter(x=df_simu['Close_rel'], y=df_simu['Close_rel'], mode='lines')

    return fig, fig2


#if __name__ == '__main__': #VIC
app.run_server(mode='inline', debug=False) #VIC



 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Oct/2020 23:20:14] "GET /_alive_dc618a58-1807-46e3-83bd-fbbed4ec8fc9 HTTP/1.1" 200 -


In [1]:
%load_ext watermark

%watermark -v -m -p dash,dash_html_components,dash_core_components,dash_bootstrap_components,dash.dependencies,jupyter_dash,plotly.express,pandas,datetime,re,watermark

print(" ")
%watermark -u -n -t -z

CPython 3.7.6
IPython 7.12.0

dash 1.16.1
dash_html_components 1.0.1
dash_core_components 1.3.1
dash_bootstrap_components 0.10.7
dash.dependencies 1.16.1
jupyter_dash 0.3.0
plotly.express 4.10.0
pandas 1.0.1
datetime unknown
re 2.2.1
watermark 2.0.2

compiler   : MSC v.1916 64 bit (AMD64)
system     : Windows
release    : 10
machine    : AMD64
processor  : Intel64 Family 6 Model 158 Stepping 9, GenuineIntel
CPU cores  : 8
interpreter: 64bit
 
last updated: Sat Oct 31 2020 15:06:28 Romance Standard Time


In [4]:
#import watermark

In [5]:
#import IPython
#print(IPython.sys_info())